---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [61]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [6]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    state = ''
    table = []
    row = []
    with open('university_towns.txt') as f:
        for line in f:
            if line.find('[edit]') > 0:
                state = line[:-7]
            else:
                row.append(state)
                if line.find('(') > -1:
                    offset = line.find('(') - 1
                    row.append(line[:offset])
                elif line.find(':') > -1:
                    offset = line.find(':')
                    row.append(line[:-1])
                else:
                    row.append(line.rstrip())
                table.append(row)
                row = []
                
    df = pd.DataFrame(table, columns=['State', 'RegionName'])
    return df

In [8]:
def get_recession_quarters():
        
    df = pd.read_excel(open('gdplev.xls','rb'), skiprows=219, sheetname='Sheet1')
    df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=True)
    df.columns = ['Quarter', 'Chained']

    quarters = []
    end = []
    inRecession = False
    for i in range(1, len(df) - 1):
        prev = df.iloc[i - 1].Chained
        current = df.iloc[i].Chained
        nxt = df.iloc[i + 1].Chained
        if prev > current and (current > nxt or inRecession):
            if current > nxt:
                inRecession = True
            else:
                end.append(df.iloc[i].Quarter)
                inRecession = False
            quarters.append({
                'quarter':df.iloc[i].Quarter,
                'chained':df.iloc[i].Chained,
            })

    return { 'quarters': quarters, 'end': end }

def get_recession_start():
    return get_recession_quarters()['quarters'][0]['quarter']


get_recession_start()

'2008q3'

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    end = get_recession_quarters()['end']
    return end[len(end) - 1]

get_recession_end()

'2009q2'

In [10]:
from operator import itemgetter
def get_recession_bottom():
    quarters = get_recession_quarters()['quarters']
    return sorted(quarters, key=itemgetter('chained'))[0]['quarter']

get_recession_bottom()

'2009q2'

In [40]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    quarter = {
        '01': 'q1',
        '04': 'q2',
        '07': 'q3',
        '10': 'q4'
    }
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    yearMonths = [col for col in df.columns if col.find('-') > 0]
    wantedYearMonths = [col for col in yearMonths if col.find('199') < 0]
    quartersDf = df[wantedYearMonths].groupby(pd.PeriodIndex(wantedYearMonths, freq='Q'), axis=1).mean()
    quartersDf.columns = quartersDf.columns.strftime('%Yq%q')
    df.drop(yearMonths + ['RegionID', 'Metro', 'CountyName', 'SizeRank'], inplace=True, axis=1)
    df = pd.concat([df, quartersDf], axis=1)
    df['State'] = df['State'].apply(lambda x: states[x])
    df = df.set_index(['State', 'RegionName']) 
    return df

In [71]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
#     convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]
    prices = convert_housing_data_to_quarters()
    recStart = prices[get_recession_start()]
    recBottom = prices[get_recession_bottom()]
    prices['growth'] = recStart / recBottom
    
    universityTowns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    collegeGrowth = prices.loc[list(universityTowns.index)]['growth'].dropna()
    nonCollegeIndexs = set(prices.index) - set(collegeGrowth.index)
    nonCollegeGrowth = prices.loc[list(nonCollegeIndexs)]['growth'].dropna()
    statistic, pValue = tuple(ttest_ind(collegeGrowth, nonCollegeGrowth))
    
    better = ["non-university town", "university town"]
    
    return (pValue < 0.05, pValue, better[statistic < 0])

run_ttest()

(True, 0.005496427353694603, 'university town')